In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd

import rasterio as rio
import rioxarray
import rasterstats as rs
from rasterstats import zonal_stats
import xarray as xr

import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning

warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning)

### Exploring Land Surface Temperature

ESA Land Surface Temperature Climate Change Initiative (LST_cci): Monthly Multisensor Infra-Red (IR) Low Earth Orbit (LEO) land surface temperature (LST) time series level 3 supercollated (L3S) global product (1995-2020), version 2.00. More information from the CEDA Archive can be found [here](https://catalogue.ceda.ac.uk/uuid/785ef9d3965442669bff899540747e28).

Now let's explore Land Surface Temperature, but this time let's try to access directly from the URL

In [ ]:
# must add `#mode=bytes` to the end (see: https://github.com/Unidata/netcdf4-python/issues/1043)
# url = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/12/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201201000000-fv2.00.nc#mode=bytes"
url = "https://dap.ceda.ac.uk/neodc/esacci/land_surface_temperature/data/MULTISENSOR_IRCDR/L3S/0.01/v2.00/monthly/2020/11/ESACCI-LST-L3S-LST-IRCDR_-0.01deg_1MONTHLY_DAY-20201101000000-fv2.00.nc#mode=bytes"
# can check status of CEDA core archives here: https://stats.uptimerobot.com/vZPgQt7YnO, currently `dap` is down

In [ ]:
ds_disk = xr.open_dataset(url, mask_and_scale=True)
ds_disk

Before we do any processing of the data, we will want to clip the global dataset down to our AOI (continent of Africa) to reduce the size and processing time. To do this, we'll import the national administrative boundaries (admin 0) for Africa in order to create a bounding box for the continent.

In [ ]:
## Hardcoded bounding box from admin0 boundaries instead and save time reading in file

# import geopandas as gpd

# admin0_gdf = gpd.read_file(
#     "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_0&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
# )
# # admin0_gdf

# print(admin0_gdf.crs)

In [ ]:
# xmin, ymin, xmax, ymax = admin0_gdf.geometry.total_bounds
# print(xmin, ymin, xmax, ymax)

In [ ]:
# sorting solves CRS issue? see: https://gis.stackexchange.com/questions/396365/using-rioxarray-to-assign-spatial-reference-epsg4326-to-netcdf-built-from-csv
ds_sort = ds_disk.sortby(["time", "lat", "lon"])

In [ ]:
# Assign crs to match boundaries
ds_sort.rio.write_crs("epsg:4326", inplace=True)

In [ ]:
# Subset to bounding box of African continent
lst_clip = ds_sort["lst"].rio.clip_box(
    minx=-25.35874748,
    miny=-46.9813795,
    maxx=63.5026492,
    maxy=37.560954,
    crs="EPSG:4326",
)

In [ ]:
lst_clip.dims
lst_clip.coords

In [ ]:
# Convert from Kelvin to Celsius
lst_africa_c = lst_clip - 273.15
lst_africa_c

In [ ]:
lst_africa_c.plot()

### Admin 2 (District) boundary files
We'll access those from ICPAC, in the future - these will be replaced with the shapefile from the John Hopkins repo. 

In [ ]:
admin2_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_2&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
admin2_gdf

In [ ]:
admin2_gdf.crs

In [ ]:
admin2_gdf.boundary.plot(alpha=0.2, color="black")

### Running Zonal Statistics

Now let's plot the two layers on top of each other to make sure that they overlap. We'll also look at the extent of missing data on the map. 

In [ ]:
ax = admin2_gdf.boundary.plot(alpha=0.2, color="black")
lst_africa_c.plot(ax=ax, zorder=-1)

In [ ]:
# correcting the affine, so they are not negative (previous error without this resulted in 'no negative dimensions allowed'admin2_gdf)
affine = lst_africa_c.rio.transform()
gdal = list(affine.to_gdal())
gdal[-1] = -gdal[-1]
affine = affine.from_gdal(*gdal)
affine

In [ ]:
admin2_lst = rs.zonal_stats(
    admin2_gdf["geometry"],
    lst_africa_c.squeeze().values,
    affine=affine,  # using corrected affine above
    stats="mean",
    nodata=-999,  # eliminate "nodata" warnings
    geojson_output=True,
    copy_properties=True,
)
admin2_lst

### Trouble-shooting: 

* Issue with `mean`: `None` is likely due to a multigeometry problem, where a solution is not yet implemented in `rasterstats`. See this PR [here](https://github.com/perrygeo/python-rasterstats/pull/255).  Will need to test with single polygon.
* Or does the affine transformation introduce some offset that means the crs are not the same? 

### Trying with Admin 0 (National) Boundaries

In [ ]:
import geopandas as gpd

admin0_gdf = gpd.read_file(
    "https://geoportal.icpac.net/geoserver/ows?service=WFS&version=1.0.0&request=GetFeature&typename=geonode%3Aafr_g2014_2013_0&outputFormat=json&srs=EPSG%3A4326&srsName=EPSG%3A4326"
)
# admin0_gdf

In [ ]:
print(admin0_gdf.crs)

Plotting both the national boundaries and LST against each other. 

In [ ]:
ax = admin0_gdf.boundary.plot(alpha=0.2, color="black")
lst_africa_c.mean(dim="time").plot(ax=ax, zorder=-1)

In [ ]:
admin0_lst = rs.zonal_stats(
    admin0_gdf["geometry"],
    lst_africa_c.squeeze().values,
    affine=affine,  # using corrected affine above
    stats="mean",
    nodata=-999,  # eliminate "nodata" warnings
    geojson_output=True,
    copy_properties=True,
)
admin0_lst